<a href="https://colab.research.google.com/github/ajhilins/covid-data-analysis/blob/main/covidForecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import RepeatedKFold,cross_val_score
import altair as alt

#Covid-19 Forecast in Indiana

The goal of this project is to predict the number of confirmed cases in the days ahead of us using regression. I will be using data gathered by Johns Hopkins that is based on the number of confirmed cases so far. I decided to limit my area of interest to the counties of Indiana.

###Preprocessing Data

The data consists of 11 attributes excluding the time series data.


*   UID: uinique id
*   iso2: country identifier
*   iso3: country identifier
*   code3: country identifier
*   FIPS: unique id for counties in US
*   Admin2: county name
*   Province_State: state name
*   Country_region: country name
*   Lat: latitude
*   Long_: longitude
*   Combined_Key: county, state, country combination
*   The other attributes are confirmed cases on that date.

I am going to filter out all of the attributes aside from the confirmed cases and the Admin2 attribute for clarity.



In [15]:
#the data was acquired from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
myData = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")

#acquired from http://www.stats.indiana.edu/population/popTotals/2017_cntyest.asp
countyPop = pd.read_csv("indianaCountyPopulation.csv")
#filtering out data not from indiana
myData = myData.loc[myData["Province_State"] == "Indiana"]
#lets filter out irrelevant location data since we are focused on counties in indiana
myData = myData.drop(columns=["UID","iso2","iso3","code3","FIPS","Province_State","Country_Region","Lat","Long_","Combined_Key"])
#transforming data for time series analysis
counties = myData.Admin2.values
transposedData = myData.loc[myData["Admin2"]=="Adams"].T
for county in counties:
  transposedData = pd.concat([transposedData, myData.loc[myData["Admin2"]==county].T], axis=1)
myData = transposedData.T.drop_duplicates().T

#cleaning up indices and removing extraneous data
myData = myData.drop(myData.index[0])
counties = np.delete(counties,[92,93])
myData = myData.iloc[:,0:92]
myData.columns = counties

#normalizing population data
for index,county in enumerate(counties):
    for j in range(0,len(myData["Adams"])):
      myData[county][j] = (myData[county][j]/countyPop.iat[index,1])*1000

#dropping all data before 3/5 since its the day before the first confirmed case
myData = myData.drop(myData.index[0:43])

#Visualization

This was going to be an interactive plot, but currently google colab does not support matplotlibs interactive plotting.

In [ ]:
fig, ax = plt.subplots()
for county in counties:
  ax.plot(myData[county])
ax.legend(counties,ncol=4,bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
fig.subplots_adjust(right=0.55)
plt.xticks(rotation=90)
plt.title("Number of Confirmed Cases by County in Indiana Per Day")
plt.xlabel("Date Confirmed")
plt.ylabel("Confirmed Cases(Normalized by Pop.)")
ax.xaxis.set_major_locator(ticker.MaxNLocator(8))
plt.show()

###Modelling

Since most of our data appears to be linear although we know that is not the case in a pandemic, I will first be modelling using a simple linear regression model.

In [18]:
print()
numberOfDays = np.arange(1,len(myData["Adams"])+1)
myData['numberOfDays'] = numberOfDays

####Linear Model

In [ ]:
def linearModel(county):
  n = len(myData[county])
  X = np.array(myData["numberOfDays"].values)
  X = X.reshape(-1,1)
  y = np.array(myData[county].values)


  # Split the data into training/testing sets
  X_train = X[:-20]
  X_test = X[-20:]
  y_train = y[:-20]
  y_test = y[-20:]

  # Create linear regression object
  lm = linear_model.LinearRegression()

  # Train the model using the training sets
  lm.fit(X_train, y_train)

  # Make predictions using the testing set
  y_pred = lm.predict(X_test)

  # The mean squared error
  print('Mean squared error: {}'.format(mean_squared_error(y_test, y_pred)))
  # The coefficient of determination: 1 is perfect prediction
  print('Coefficient of determination: {}'.format(r2_score(y_test, y_pred)))
  # predict ten days in advance
  for x in range(1,11):
    # prediction
    print('Predicted: {}'.format(lm.predict(np.array([n+x]).reshape(-1,1))))

  # Plot outputs
  #plt.scatter(X_test, y_test,  color='black')
  #plt.plot(X_test, y_pred, color='blue', linewidth=3)
  #plt.xticks(())
  #plt.yticks(())
  #plt.show()



for county in counties:
  linearModel(county)
  print()

After reviewing the erratic mean squared error(which tells us how close our regression line is to the points) and r-squared which is more beneficial because we scaled the data, we can see that the linear model just isnt cutting it. Even though on the inital analysis it looked as though the data had linear trends it is definetly not linear. Lets see if scaling the data to have a mean 0 and variance of 1 changes anything.

In [ ]:
def ridgeRegression(county):
  n = len(myData[county])
  X = np.array(myData["numberOfDays"].values)
  X = X.reshape(-1,1)
  y = np.array(myData[county].values)

  # Create ridge regression object
  rm = Ridge()

  # Train the model using the training sets
  rm.fit(X,y)

  # KFold for cross validation
  cv = RepeatedKFold(n_splits=10,n_repeats=3,random_state=5)

  # Eval of model performance
  scores = cross_val_score(rm, X, y, scoring='r2', cv=cv, n_jobs=-1)
  print('Scores: {}'.format(scores))

  for x in range(1,11):
    print('Predicted: {}'.format(rm.predict(np.array([n+x]).reshape(-1,1))))



  # Plot outputs
  #plt.scatter(X, y,  color='black')
  #plt.plot(X, rm.predict(X), color='blue', linewidth=3)
  #plt.xticks(())
  #plt.yticks(())
  #plt.show()



for county in counties:
  ridgeRegression(county)
  print()

#Conclusion

The Covid19 outbreak is difficult to use a general model for because of the widely varying circumstances that every region has. This model is not good at predicting cases because it cannot take into account the other extremely important factors such as infection rate, immunity rate, fatality rate and recovery rate. The models generated here for each county seem to be underfit because of these additional factors the dataset did not contain. With all that being said these models function well due to the fact that the number of cases continues to climb, when the number of cases ceases to grow and the line stagnates the models will begin to perform worse.

##Predictions
For the predictions of each county we can see a variety of things happening, the counties that are larger we can easily see the sharp upturn in cases, the smaller the county size the more gradual the slope. Meaning the larger the county is the faster the spread is which makes sense.